# Chapter 1

- Response variable = dependent variable = y
- Explanatory variables = independent variables = X
- Linear regression = The response variable is numeric
- Logistic regression = The response variable is logical.
- Simple linear/logistic regression = There is only one explanatory variable


```
# You want a regression line : y = mx + c
from statsmodels.formula.api import ols
# Formula applied : y = x, finds m and c on its own for best fit
num_model = ols("y ~ x_num", data=df).fit()
# Calculate co-efficient of each category with relative to 0 instead of relative to intercept, its the mean for each category
cat_model = ols("y ~ x_cat + 0", data=df).fit()
# See model params : slope m and intercept c
print(cat_model.params)
```

# Chapter 2